# Import Libraries

In [71]:
from datetime import datetime
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import yfinance as yf
from numpy import mat
from sklearn.ensemble import RandomForestRegressor, VotingClassifier, GradientBoostingRegressor, VotingRegressor
from finta import TA

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
import xgboost as xgb

In [72]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [73]:
# Define the ticker symbols and period of interest
ticker = "AAPL"
period = "1y"   

# Create an empty DataFrame with column names
df_predicts = pd.DataFrame(columns=['Stock Name', 'Actual', 'Prediction'])
df_companies = pd.DataFrame(columns=['Stock', 'Open', 'High', 'Low', 'Close', 'Volume'])

data = yf.download(ticker, period=period)
# Remove the current month's data
current_month = datetime.now().month
current_year = datetime.now().year

# Filter out the data for the current month and year
data = data[~((data.index.month == current_month) & (data.index.year == current_year))]
# Preprocessing
data = data.copy()
data.rename(columns={"Close": 'close', "High": 'high', "Low": 'low', 'Volume': 'volume', 'Open': 'open'},
            inplace=True)

# List of symbols for technical indicators
INDICATORS = ['RSI', 'MACD', 'STOCH', 'ADL', 'ATR', 'MOM', 'MFI', 'ROC', 'OBV', 'CCI', 'EMV', 'VORTEX']

def _get_indicator_data(data):
    """
    Function that uses the finta API to calculate technical indicators used as the features
    :return:
    """

    for indicator in INDICATORS:
        ind_data = eval('TA.' + indicator + '(data)')
        if not isinstance(ind_data, pd.DataFrame):
            ind_data = ind_data.to_frame()
        data = data.merge(ind_data, left_index=True, right_index=True)
    data.rename(columns={"14 period EMV.": '14 period EMV'}, inplace=True)

    # Also calculate moving averages for features
    data['ema50'] = data['close'] / data['close'].ewm(50).mean()
    data['ema21'] = data['close'] / data['close'].ewm(21).mean()
    data['ema15'] = data['close'] / data['close'].ewm(14).mean()
    data['ema5'] = data['close'] / data['close'].ewm(5).mean()

    # Instead of using the actual volume value (which changes over time), we normalize it with a moving volume average
    data['normVol'] = data['volume'] / data['volume'].ewm(5).mean()

    # Remove columns that won't be used as features
    del (data['open'])
    del (data['high'])
    del (data['low'])
    del (data['volume'])
    del (data['Adj Close'])

    return data


data = _get_indicator_data(data)

df = pd.DataFrame(data)

df_companies = df_companies._append(df)
print("taskin")


[*********************100%%**********************]  1 of 1 completed
taskin


In [74]:
df

,close,14 period RSI,MACD,SIGNAL,14 period STOCH %K,MFV,14 period ATR,MOM,14 period MFI,ROC,OBV,20 period CCI,14 period EMV,VIm,VIp,ema50,ema21,ema15,ema5,normVol
Date,,,,,,,,,,,,,,,,,,,,
2022-12-27,130.029999,NaN,0.000000,0.000000,NaN,-1.795928e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
2022-12-28,126.040001,0.000000,-0.089519,-0.049733,NaN,-8.160472e+07,NaN,NaN,NaN,NaN,-8.543840e+07,-66.666667,NaN,NaN,NaN,0.984570,0.984775,0.984948,0.985815,1.095786
2022-12-29,129.610001,49.072176,-0.003360,-0.030727,NaN,-5.380068e+07,NaN,NaN,NaN,NaN,-9.734700e+06,20.991683,NaN,NaN,NaN,1.008188,1.008211,1.008227,1.008258,0.982229
2022-12-30,129.929993,51.375455,0.047463,-0.004240,NaN,2.201049e+07,NaN,NaN,NaN,NaN,6.729950e+07,8.176198,NaN,NaN,NaN,1.007906,1.007814,1.007731,1.007262,0.999655
2023-01-03,125.070000,29.530725,-0.119311,-0.038471,NaN,-6.012009e+07,NaN,NaN,NaN,NaN,-4.481800e+07,-111.554693,NaN,NaN,NaN,0.976256,0.976508,0.976725,0.977873,1.291240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-24,189.970001,66.996178,4.097630,3.303982,82.296687,1.315069e+09,2.549284,7.559998,77.775521,4.482452,1.648920e+09,77.439698,0.043187,0.487252,1.312132,1.058954,1.047441,1.039098,1.011307,0.528792
2023-11-27,189.789993,66.399874,3.981732,3.439532,77.507154,1.315298e+09,2.445714,3.389999,71.191564,3.772756,1.608367e+09,69.438494,0.032429,0.548481,1.231600,1.056737,1.044244,1.035483,1.008609,0.908092
2023-11-28,190.399994,67.456934,3.894215,3.530468,77.689598,1.322615e+09,2.317857,5.599991,69.921530,4.380237,1.646783e+09,69.980988,0.023712,0.603698,1.174730,1.058872,1.045338,1.036130,1.009856,0.880750


In [75]:
print(df_companies.index)

DatetimeIndex(['2022-12-27', '2022-12-28', '2022-12-29', '2022-12-30',
               '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06',
               '2023-01-09', '2023-01-10',
               ...
               '2023-11-16', '2023-11-17', '2023-11-20', '2023-11-21',
               '2023-11-22', '2023-11-24', '2023-11-27', '2023-11-28',
               '2023-11-29', '2023-11-30'],
              dtype='datetime64[ns]', length=234, freq=None)


In [76]:
data

,close,14 period RSI,MACD,SIGNAL,14 period STOCH %K,MFV,14 period ATR,MOM,14 period MFI,ROC,OBV,20 period CCI,14 period EMV,VIm,VIp,ema50,ema21,ema15,ema5,normVol
Date,,,,,,,,,,,,,,,,,,,,
2022-12-27,130.029999,NaN,0.000000,0.000000,NaN,-1.795928e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
2022-12-28,126.040001,0.000000,-0.089519,-0.049733,NaN,-8.160472e+07,NaN,NaN,NaN,NaN,-8.543840e+07,-66.666667,NaN,NaN,NaN,0.984570,0.984775,0.984948,0.985815,1.095786
2022-12-29,129.610001,49.072176,-0.003360,-0.030727,NaN,-5.380068e+07,NaN,NaN,NaN,NaN,-9.734700e+06,20.991683,NaN,NaN,NaN,1.008188,1.008211,1.008227,1.008258,0.982229
2022-12-30,129.929993,51.375455,0.047463,-0.004240,NaN,2.201049e+07,NaN,NaN,NaN,NaN,6.729950e+07,8.176198,NaN,NaN,NaN,1.007906,1.007814,1.007731,1.007262,0.999655
2023-01-03,125.070000,29.530725,-0.119311,-0.038471,NaN,-6.012009e+07,NaN,NaN,NaN,NaN,-4.481800e+07,-111.554693,NaN,NaN,NaN,0.976256,0.976508,0.976725,0.977873,1.291240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-24,189.970001,66.996178,4.097630,3.303982,82.296687,1.315069e+09,2.549284,7.559998,77.775521,4.482452,1.648920e+09,77.439698,0.043187,0.487252,1.312132,1.058954,1.047441,1.039098,1.011307,0.528792
2023-11-27,189.789993,66.399874,3.981732,3.439532,77.507154,1.315298e+09,2.445714,3.389999,71.191564,3.772756,1.608367e+09,69.438494,0.032429,0.548481,1.231600,1.056737,1.044244,1.035483,1.008609,0.908092
2023-11-28,190.399994,67.456934,3.894215,3.530468,77.689598,1.322615e+09,2.317857,5.599991,69.921530,4.380237,1.646783e+09,69.980988,0.023712,0.603698,1.174730,1.058872,1.045338,1.036130,1.009856,0.880750


In [78]:
def _train_xgboost(X_train, y_train, X_test, y_test):
    # Define model
    xgb_model = xgb.XGBRegressor()

    # Define hyperparameters to tune
    param_grid = {
        'n_estimators': [100, 500, 1000],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.5],
    }

    # Perform grid search
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    # Save best model
    xgb_best = grid_search.best_estimator_
    # Make predictions on the testing data
    y_pred = xgb_best.predict(X_test)
    return xgb_best


def _train_random_forest(X_train, y_train, X_test, y_test):
    """
    Function that uses random forest classifier to train the model
    :return:
    """

    # Define parameter grid for GridSearchCV
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Create Random Forest Regressor
    rf = RandomForestRegressor()

    # Use GridSearchCV to find best hyperparameters
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Save best model
    rf_best = grid_search.best_estimator_
    # Make predictions on the testing data
    y_pred = rf_best.predict(X_test)

    return rf_best


def _train_svr(X_train, y_train, X_test, y_test):
    svr = SVR(kernel='rbf')

    # Define the parameter grid for Grid Search
    param_grid = {'C': [0.1, 1, 10, 100],
                  'gamma': [1, 0.1, 0.01, 0.001],
                  'epsilon': [0.1, 0.01, 0.001, 0.0001]}

    # Perform Grid Search to find the best hyperparameters
    grid_search = GridSearchCV(svr, param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters from Grid Search
    best_params = grid_search.best_estimator_

    # Make predictions on the testing set
    predictions = best_params.predict(X_test)

    return best_params


def _ensemble_model(rf_model, knn_model, xgb_model, X_train, y_train, X_test, y_test):
    # Create a dictionary of our models
    estimators = [('svr', knn_model), ('rf', rf_model), ('xgb', xgb_model)]

    # Create our voting classifier, inputting our models
    ensemble = VotingRegressor(estimators)

    # fit model to training data
    ensemble.fit(X_train, y_train)

    # test our model on the test data
    print(ensemble.score(X_test, y_test))

    prediction = ensemble.predict(X_test)

    return ensemble

In [79]:
df = df[~((df.index.month == 12) & (df.index.year == datetime.now().year))]
train_data, test_data = train_test_split(df, test_size=0.25, shuffle=False)

train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

# Prepare the training and test data
X_train = train_data.drop(['close'], axis=1)
y_train = train_data['close']

y_test = test_data['close']
X_test = test_data.drop(['close'], axis=1)

rf_model = _train_random_forest(X_train, y_train, X_test, y_test)
svr_model = _train_svr(X_train, y_train, X_test, y_test)
xgb_model = _train_xgboost(X_train, y_train, X_test, y_test)
ensemble_model = _ensemble_model(rf_model, svr_model, xgb_model, X_train, y_train, X_test, y_test)

rf_prediction = rf_model.predict(X_test)
svr_prediction = svr_model.predict(X_test)
xgb_prediction = xgb_model.predict(X_test)
ensemble_prediction = ensemble_model.predict(X_test)

print('rf prediction is ', rf_prediction)
print('svr prediction is ', svr_prediction)
print('ensemble prediction is ', ensemble_prediction)
print('truth values are ', y_test.values)


0.6913374059841635
rf prediction is  [183.04440455 187.90665399 180.96832392 179.00526244 180.08744434
 178.68118261 179.3745623  180.58569865 179.0568947  178.14383631
 177.59948903 178.89118252 173.80315692 170.80193638 173.6616504
 171.10912495 176.78294576 175.94633101 179.0049051  180.65986565
 181.06190802 182.10150731 181.51177341 187.95117988 188.0678659
 182.23279059 188.0065175  186.35851593 183.68774285 179.32474999
 178.59301409 179.0568947  179.99444206 179.08867218 177.78313316
 178.86949042 179.7773318  180.24753344 184.36869376 192.02740771
 188.87523588 192.45980505 193.7170755  193.75444924 193.83426853
 193.90919059 193.78124256 193.73674058 193.81108559 193.68380057
 193.75814558 193.6944055  193.6921977  193.6921977  193.80094177
 193.84493373 193.64566897 193.85658277 193.7866547 ]
svr prediction is  [166.96680002 166.96680002 166.96680002 166.96680002 166.96680002
 166.96680002 166.96680002 166.96680002 166.96680002 166.96680002
 166.96680002 166.96680002 166.966

In [81]:
# Create a DataFrame with actual and predicted values
december_dates = data.loc[(data.index.year == datetime.now().year) & (data.index.month == 12)].index.date

In [82]:
december_dates

array([], dtype=object)

In [80]:
from sklearn.metrics import r2_score
r2_score(y_test, ensemble_prediction)

0.6913374059841635